In [130]:
%%javascript
require.config({
    paths: {
        flot: '//www.flotcharts.org/javascript/jquery.flot.min',
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
    }
});

<IPython.core.display.Javascript object>

In [131]:
%%html
<style>
div.output {
  max-height: 1000px;
  overflow-y: scroll;
}
</style>

In [132]:
import numpy as np
import json
import time
from IPython.display import Javascript
from IPython.core.display import display

In [134]:
# sigmoid function w/ derivative option
def sigmoid(x, deriv=False):
    if (deriv):
        return x*(1-x)
    return 1/(1 + np.exp(-x))

# inputs
X = np.array([[0,0,1],
             [0,1,1],
             [1,0,1],
             [1,1,1]])

# outputs
y = np.array([[0,1,1,0]]).T

# seed random numbers
np.random.seed(3)

# initialize random weights
W0 = 2*np.random.random((3,4)) - 1
W1 = 2*np.random.random((4,1)) - 1

js = Javascript("""
    element.append("<h2 id='chart-header' style='text-align: center;'>Visualize Error & Weights Changing Over Time</h2><br/><br />")
    element.append("<div id='chart' style='position: relative; width:90%; height: 400px; margin-top: -30px; margin-left: 20px;;'></div>")
    element.append("<div id='weight_charts' style='width: 100%; height: 320px; position: relative; padding-top:30px;'>" +
                     "<div id='weights_l0' style='position: absolute; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                     "<div id='weights_l1' style='position: absolute; left:31%; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                     "<div id='outputs' style='position: absolute; left:62%; width:27%; height: 250px; margin-left: 20px; padding-bottom: 30px;'></div>" +
                   "</div>")

    window.error = [];
    window.weights_l0 = [];
    window.weights_l1 = [];
    window.outputs = [];

    require(['flot'], function() {
      window.plot = $.plot('#chart', [[]], { series: { shadowSize: 0 }, yaxis: { min: 0, max: 0.05 }, xaxis: { min: 0, max: 20000 } })
      var yaxisLabel = $("<div class='axisLabel yaxisLabel'></div>").text("Error").appendTo($('#chart'));
      yaxisLabel.css("position", "absolute").css("font-size","16px").css("margin-top", yaxisLabel.width() / 2 - 20);
      yaxisLabel.css("top", "50%").css("left", "-30px").css("transform","rotate(-90deg)");
      var xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Iteration").appendTo($('#chart'));
      xaxisLabel.css("position", "absolute").css("width", "100px").css("font-size","16px").css("text-align","center");
      xaxisLabel.css("left", "-50px").css("bottom", "-20px").css("margin-left","50%");

      var w0_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Layer 0 Weights").appendTo($('#weight_charts'));
      w0_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      w0_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "16%");

      var w1_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Layer 1 Weights").appendTo($('#weight_charts'));
      w1_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      w1_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "47%");
      
      var outputs_xaxisLabel = $("<div class='axisLabel xaxisLabel'></div>").text("Outputs").appendTo($('#weight_charts'));
      outputs_xaxisLabel.css("position", "absolute").css("font-size","16px").css("width","160px");
      outputs_xaxisLabel.css("left", "-50px").css("bottom", "20px").css("margin-left", "81%");

      window.weight_plot_l0 = $.plot('#weights_l0', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -400, max: 400 }, xaxis: { min: 0, max: 13}})
      window.weight_plot_l1 = $.plot('#weights_l1', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -50, max: 50 }, xaxis: { min: 0, max: 5}})
      window.output_plot = $.plot('#outputs', [[]], { series: { shadowSize: 0, bars: { show: true, align: "center", barWidth: 0.5 }}, yaxis: { min: -0.5, max: 2 }, xaxis: { min: 0, max: 5}})
   
      $('#chart-header')[0].scrollIntoView();
   });
    
    
""")
display(js)

# train
for i in range(20000):
    # feed forward
    l0 = X
    l1 = sigmoid(np.dot(l0,W0))
    l2 = sigmoid(np.dot(l1,W1))
    
    # find error
    l2_error = y - l2
        
    # calculate update
    l2_delta = l2_error*sigmoid(l2)
    
    # backpropagate
    l1_error = l2_delta.dot(W1.T)
    
    # calculate update
    l1_delta = l1_error*sigmoid(l1)
    
    # update weights
    W1 += l1.T.dot(l2_delta)
    W0 += l0.T.dot(l1_delta)
    
    if (i % 50 == 0):
        weights_l0 = [[idx+1, weight] for idx, weight in enumerate(W0.flatten())]
        weights_l1 = [[idx+1, weight] for idx, weight in enumerate(W1.flatten())]
        outputs = [[idx+1, guess] for idx, guess in enumerate(l2.flatten())]
        display(Javascript("window.error.push([%d,%.5f])" % (i,np.mean(np.abs(l2_error)))));
        display(Javascript("window.plot.setData([window.error]); window.plot.draw();"));
        display(Javascript("window.weights_l0 = %s" % json.dumps(weights_l0)));
        display(Javascript("window.weight_plot_l0.setData([window.weights_l0]); window.weight_plot_l0.draw();"));
        display(Javascript("window.weights_l1 = %s" % json.dumps(weights_l1)));
        display(Javascript("window.weight_plot_l1.setData([window.weights_l1]); window.weight_plot_l1.draw();"));
        display(Javascript("window.outputs = %s" % json.dumps(outputs)));
        display(Javascript("window.output_plot.setData([window.outputs]); window.output_plot.draw();"));
        time.sleep(0.1)
    
results = zip(y,l2)
#[(print("Actual: %.0f  Guess: %.5f" % (result[0], result[1]))) for result in results]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>